In [76]:
import os
import sys
import copy
import wget
import time
import asyncio
import warnings
import logging

import numpy as np

from astropy import time as astropytime
from lsst.ts import salobj
from lsst.ts.standardscripts.auxtel.attcs import ATTCS
from lsst.ts.standardscripts.auxtel.latiss import LATISS

%matplotlib inline

Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [2]:
stream_handler = logging.StreamHandler(sys.stdout)

logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [3]:
attcs = ATTCS()
# cam = salobj.Remote(attcs.domain, "GenericCamera", 1)

await attcs.start_task

Read historical data in 2.12 sec
Read historical data in 2.57 sec
Read historical data in 3.69 sec
Read historical data in 4.46 sec
Read historical data in 5.53 sec
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 79 messages
Read historical data in 6.40 sec
Read historical data in 7.36 sec
RemoteTelemetry(ATDome, 0, position) falling behind; read 23 messages


[None, None, None, None, None, None, None]

RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 15 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 16 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 16 messages


In [78]:
latiss = LATISS(attcs.domain)

RemoteEvent(ATMCS, 0, target) falling behind; read 100 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 38 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 37 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 39 messages
Read historical data in 0.59 sec
Read historical data in 0.93 sec
Read historical data in 1.25 sec
Read historical data in 1.88 sec
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 52 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 10 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 11 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 11 messages
RemoteEvent(ATDomeTrajectory, 0, heartbeat) falling behind; read 11 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 10 messages
RemoteEvent(ATDome, 0, heartbeat) falling behind; read 11 messages
R

In [79]:
await latiss.start_task

[None, None, None, None]

In [4]:
# Declare target name
target_name='HD107696'

RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 17 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 38 messages


In [ ]:
await attcs.enable(settings={
    'ataos': "current",
    'atmcs': "",
    'atptg': "",
    'atpneumatics': "",
    'athexapod': "auxtel_201912_crazy.yaml",
    'atdome': "test.yaml",
    'atdometrajectory': ""})

In [ ]:
rotation_matrix = lambda angle: np.array([
            [np.cos(np.radians(angle)), -np.sin(np.radians(angle)), 0.],
            [np.sin(np.radians(angle)), np.cos(np.radians(angle)), 0.],
            [0., 0., 1.]])

In [209]:
await attcs.slew_icrs(ra = "12 22 49.43", dec = "-57 40 34.1", rot_pa=45., target_name=target_name)

Flushing events
Sending command
Scheduling check coroutines
process as completed...
[Telescope] delta Alt = -000.049 | delta Az= +000.038 [Dome] delta Az = +000.247


AttributeError: 'NoneType' object has no attribute 'private_sndStamp'

[Telescope] delta Alt = +000.002 | delta Az= +000.001 [Dome] delta Az = +000.247
Axes in position.


In [6]:
await attcs.atdome.cmd_openShutter.start()

[Telescope] delta Alt = -000.001 | delta Az= +163.230 [Dome] delta Az = -067.956
[Telescope] delta Alt = -000.001 | delta Az= +159.229 [Dome] delta Az = -061.086
[Telescope] delta Alt = -000.000 | delta Az= +155.228 [Dome] delta Az = -054.216
[Telescope] delta Alt = -000.000 | delta Az= +151.228 [Dome] delta Az = -047.956
[Telescope] delta Alt = -000.001 | delta Az= +147.227 [Dome] delta Az = -040.376
[Telescope] delta Alt = -000.000 | delta Az= +143.226 [Dome] delta Az = -034.186
[Telescope] delta Alt = -000.001 | delta Az= +139.226 [Dome] delta Az = -027.286
[Telescope] delta Alt = -000.001 | delta Az= +135.224 [Dome] delta Az = -020.376
[Telescope] delta Alt = -000.001 | delta Az= +131.223 [Dome] delta Az = -013.486
[Telescope] delta Alt = -000.001 | delta Az= +127.224 [Dome] delta Az = -006.546
[Telescope] delta Alt = -000.000 | delta Az= +123.223 [Dome] delta Az = -000.396
[Telescope] delta Alt = -000.000 | delta Az= +119.221 [Dome] delta Az = +002.494
[Telescope] delta Alt = -000

In [36]:
await attcs.atptg.cmd_focusName.set_start(focus=3)

In [56]:
-1./4200.*-74, -1./131*-60

(0.017619047619047618, 0.4580152671755725)

In [55]:
attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_offset.set_start(x=0.,y=0.,z=-0.04)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

In [ ]:
attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_offset.set_start(x=-0.5,y=0.4,z=0.)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

In [ ]:
await attcs.atptg.cmd_offsetAzEl.set_start(az=49.-41.22144+27, el=16.+35-21.6, num=0) 

In [54]:
-1./4200.*173

-0.04119047619047619

In [ ]:
x = -60
y = +80

In [57]:
attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_offset.set_start(x=0.,y=0.,z=0.02*0.6)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

RemoteEvent(ATMCS, 0, target) falling behind; read 49 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 20 messages
RemoteTelemetry(ATMCS, 0, measuredMotorVelocity) falling behind; read 10 messages
RemoteTelemetry(ATMCS, 0, azEl_mountMotorEncoders) falling behind; read 59 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 57 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 57 messages
RemoteTelemetry(ATMCS, 0, nasymth_m3_mountMotorEncoders) falling behind; read 57 messages
RemoteTelemetry(ATMCS, 0, mount_Nasmyth_Encoders) falling behind; read 57 messages
RemoteTelemetry(ATMCS, 0, mount_AzEl_Encoders) falling behind; read 56 messages
RemoteTelemetry(ATMCS, 0, measuredTorque) falling behind; read 55 messages
RemoteEvent(ATMCS, 0, heartbeat) falling behind; read 53 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 15 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 36 messages
Remote

In [ ]:
-54.*0.4

In [ ]:
-54*0.648854961832061

In [ ]:
await attcs.ataos.cmd_offset.set_start(x=0.,y=0.,z=-0.8)



In [48]:
await attcs.atptg.cmd_offsetAzEl.set_start(az=200., el=-30., num=0) 

In [49]:
await attcs.atptg.cmd_offsetAzEl.set_start(az=0., el=0., num=0) 

Positive azimuth is negative y
Positive elevation is posi

In [43]:
await attcs.atptg.cmd_offsetAzEl.set_start(az=160., el=40., num=0) 

In [17]:
await attcs.atptg.cmd_stopTracking.start()

If rotator is aligned with boresight:

 +El-offset = -x-axis

 +Az-offset = -y-axis
 
 hexapod movement in y is movement in x
 

In [ ]:
await attcs.atptg.cmd_offsetAzEl.set_start(az=49., el=16., num=0)

In [ ]:
azel = await attcs.atmcs.tel_mount_AzEl_Encoders.aget()
nasmyth = await attcs.atmcs.tel_mount_Nasmyth_Encoders.aget()
angle = np.mean(azel.elevationCalculatedAngle)+np.mean(nasmyth.nasmyth1CalculatedAngle)
el, az, _ = np.matmul([20, -18, 0.], rotation_matrix(angle))
print(az,el)
await attcs.atptg.cmd_offsetAzEl.set_start(az=az, el=el, num=0)


In [ ]:
azel = await attcs.atmcs.tel_mount_AzEl_Encoders.aget()
nasmyth = await attcs.atmcs.tel_mount_Nasmyth_Encoders.aget()
angle = np.mean(azel.elevationCalculatedAngle)+np.mean(nasmyth.nasmyth1CalculatedAngle)
el, az, _ = np.matmul([20-90., -18, 0.], rotation_matrix(angle))
print(az,el)
await attcs.atptg.cmd_offsetAzEl.set_start(az=az, el=el, num=0)



In [ ]:
offset_x = (np.random.random()-0.5)*240.
offset_y = (np.random.random()-0.5)*240.
print(offset_x, offset_y)
azel = await attcs.atmcs.tel_mount_AzEl_Encoders.aget()
nasmyth = await attcs.atmcs.tel_mount_Nasmyth_Encoders.aget()
angle = np.mean(azel.elevationCalculatedAngle)+np.mean(nasmyth.nasmyth1CalculatedAngle)
el, az, _ = np.matmul([20+offset_x, -18+offset_y, 0.], rotation_matrix(angle))
await attcs.atptg.cmd_offsetAzEl.set_start(az=az, el=el, num=0)


In [ ]:
c = salobj.Controller("Script", index=11)

In [ ]:
await c.start_task

In [ ]:
offset_x = (np.random.random()-0.5)*240.
offset_y = (np.random.random()-0.5)*240.
print(offset_x, offset_y)
azel = await attcs.atmcs.tel_mount_AzEl_Encoders.aget()
nasmyth = await attcs.atmcs.tel_mount_Nasmyth_Encoders.aget()
angle = np.mean(azel.elevationCalculatedAngle)+np.mean(nasmyth.nasmyth1CalculatedAngle)
el, az, _ = np.matmul([20+offset_x, -18+offset_y, 0.], rotation_matrix(angle))
await attcs.atptg.cmd_offsetAzEl.set_start(az=az, el=el, num=0)



In [ ]:
offset_x = (np.random.random()-0.5)*240.
offset_y = (np.random.random()-0.5)*240.
print(offset_x, offset_y)
azel = await attcs.atmcs.tel_mount_AzEl_Encoders.aget()
nasmyth = await attcs.atmcs.tel_mount_Nasmyth_Encoders.aget()
angle = np.mean(azel.elevationCalculatedAngle)+np.mean(nasmyth.nasmyth1CalculatedAngle)
el, az, _ = np.matmul([20+offset_x, -18+offset_y, 0.], rotation_matrix(angle))
await attcs.atptg.cmd_offsetAzEl.set_start(az=az, el=el, num=0)

In [ ]:
offset_x = (np.random.random()-0.5)*240.
offset_y = (np.random.random()-0.5)*240.
print(offset_x, offset_y)
azel = await attcs.atmcs.tel_mount_AzEl_Encoders.aget()
nasmyth = await attcs.atmcs.tel_mount_Nasmyth_Encoders.aget()
angle = np.mean(azel.elevationCalculatedAngle)+np.mean(nasmyth.nasmyth1CalculatedAngle)
el, az, _ = np.matmul([20+offset_x, -18+offset_y, 0.], rotation_matrix(angle))
await attcs.atptg.cmd_offsetAzEl.set_start(az=az, el=el, num=0)

In [ ]:
offset_x = (np.random.random()-0.5)*240.
offset_y = (np.random.random()-0.5)*240.
print(offset_x, offset_y)
azel = await attcs.atmcs.tel_mount_AzEl_Encoders.aget()
nasmyth = await attcs.atmcs.tel_mount_Nasmyth_Encoders.aget()
angle = np.mean(azel.elevationCalculatedAngle)+np.mean(nasmyth.nasmyth1CalculatedAngle)
el, az, _ = np.matmul([20+offset_x, -18+offset_y, 0.], rotation_matrix(angle))
await attcs.atptg.cmd_offsetAzEl.set_start(az=az, el=el, num=0)

In [ ]:
offset_x = (np.random.random()-0.5)*240.
offset_y = (np.random.random()-0.5)*240.
print(offset_x, offset_y)
azel = await attcs.atmcs.tel_mount_AzEl_Encoders.aget()
nasmyth = await attcs.atmcs.tel_mount_Nasmyth_Encoders.aget()
angle = np.mean(azel.elevationCalculatedAngle)+np.mean(nasmyth.nasmyth1CalculatedAngle)
el, az, _ = np.matmul([20+offset_x, -18+offset_y, 0.], rotation_matrix(angle))
await attcs.atptg.cmd_offsetAzEl.set_start(az=az, el=el, num=0)

In [ ]:
await attcs.ataos.cmd_offset.set_start(z=+0.05)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

In [ ]:
await attcs.enable(settings={
    'ataos': "current",
    'atmcs': "",
    'atptg': "",
    'atpneumatics': "",
    'athexapod': "auxtel_201912_crazy.yaml",
    'atdome': "test.yaml",
    'atdometrajectory': ""})

In [ ]:
rotation_matrix = lambda angle: np.array([
            [np.cos(np.radians(angle)), -np.sin(np.radians(angle)), 0.],
            [np.sin(np.radians(angle)), np.cos(np.radians(angle)), 0.],
            [0., 0., 1.]])
azel = await attcs.atmcs.tel_mount_AzEl_Encoders.aget()
nasmyth = await attcs.atmcs.tel_mount_Nasmyth_Encoders.aget()
angle = np.mean(azel.elevationCalculatedAngle)+np.mean(nasmyth.nasmyth1CalculatedAngle)
el, az, _ = np.matmul([20, -18., 0.], rotation_matrix(angle))

In [ ]:
el, az

In [ ]:
await attcs.atptg.cmd_offsetAzEl.set_start(az=az, el=el, num=0)

In [ ]:
5*360/24

In [ ]:
attcs.slew_icrs?

In [ ]:
await attcs.slew_icrs

In [ ]:
await attcs.ataos.cmd_applyAxisOffset.set_start(axis="x", offset=x_offset)
await attcs.ataos.cmd_applyAxisOffset.set_start(axis="y", offset=y_offset)

In [ ]:
await attcs.ataos.cmd_applyAxisOffset.set_start(axis="z", offset=focus_offset)

In [ ]:
dz = 0.8

try:
    await cam.cmd_stopLiveView.start()
except salobj.AckError:
    print("ack error")

# Intrafocal hexapod setting - offsets -0.5mm from central focus value above

print('Move to intrafocal position')
# await attcs.athexapod.cmd_moveToPosition.set_start(x=hex_X, y=hex_Y,
#                                              z=hex_Z-dz, u=hex_U, v=hex_V)
attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_applyAxisOffset.set_start(axis="z", offset=-dz+focus_offset)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

print('Take intrafocal image')
cam.evt_endReadout.flush()

await cam.cmd_takeImages.set_start(numImages=1, expTime=expTime, shutter=True, imageSequenceName=target_name+'_intra')

intra_endron = await cam.evt_endReadout.next(flush=False, timeout=5)

# Extrafocal hexapod setting
print('Move to extra-focal hexapod position')
# await attcs.athexapod.cmd_moveToPosition.set_start(x=hex_X, y=hex_Y,
#                                                      z=hex_Z+dz, u=hex_U, v=hex_V)
attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_applyAxisOffset.set_start(axis="z", offset=dz+focus_offset)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

# r.evt_endReadout.flush()
print('Take extra-focal image')
cam.evt_endReadout.flush()
await cam.cmd_takeImages.set_start(numImages=1, expTime=expTime, shutter=True, imageSequenceName=target_name+'_extra')

extra_endron = await cam.evt_endReadout.next(flush=False, timeout=5)

azel = await attcs.atmcs.tel_mount_AzEl_Encoders.aget()
nasmyth = await attcs.atmcs.tel_mount_Nasmyth_Encoders.aget()

print(f"============")
print(f"dz={dz}")
print(intra_endron.imageName)
print(extra_endron.imageName)
print(f"============")

try:
    await cam.cmd_startLiveView.set_start(expTime=0.5)
except salobj.AckError:
    print("ack error")

attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_applyAxisOffset.set_start(axis="z", offset=focus_offset)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

print(f"['{intra_endron.imageName}.fits'], ['{extra_endron.imageName}.fits']")
print(np.mean(azel.elevationCalculatedAngle)+np.mean(nasmyth.nasmyth1CalculatedAngle))

# intra_wget_url = 'http://139.229.170.216:8000/data/'+intra_endron.imageName+'.fits'
# extra_wget_url = 'http://139.229.170.216:8000/data/'+extra_endron.imageName+'.fits'
# intra_fname = wget.download(intra_wget_url)
# print(f"============")
# print(f'{intra_fname}')
# extra_fname = wget.download(extra_wget_url)
# print(f'{extra_fname}')
# print(f"============")


In [ ]:
intra_wget_url = 'http://139.229.170.216:8000/data/'+intra_endron.imageName+'.fits'
extra_wget_url = 'http://139.229.170.216:8000/data/'+extra_endron.imageName+'.fits'

intra_fname = wget.download(intra_wget_url, out="/home/saluser/develop/cwfs/python/data")
extra_fname = wget.download(extra_wget_url, out="/home/saluser/develop/cwfs/python/data")

print(intra_fname,extra_fname)

In [ ]:
await attcs.ataos.cmd_offset.set_start(z=0.)

In [ ]:
await attcs.ataos.cmd_resetOffset.start()

In [ ]:
await attcs.ataos.cmd_offset.set_start(z=-0.1)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)
print("done")

In [ ]:
await attcs.atdome.cmd_openShutter.start()

In [67]:
c = salobj.Controller("Script", index=14)

Read historical data in 0.00 sec
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 14 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 15 messages


In [68]:
await c.start_task

# Test of axis orientation on the camera.

## 1 - Take reference image with no offset. 

In [146]:
c.evt_logMessage.set_put(message="Test of axis orientation on the camera. [START]")

True

In [147]:
c.evt_logMessage.set_put(message="1 - Take reference image with no offset. [START]")

True

In [148]:
attcs.atmcs.evt_allAxesInPosition.flush()
await attcs.atptg.cmd_offsetAzEl.set_start(az=60, el=20, num=0) #refence offset
try:
    await attcs.atmcs.evt_allAxesInPosition.next(flush=False, timeout=30.)
except asyncio.TimeoutError:
    print("error getting allAxesInPosition event.")

error getting allAxesInPosition event.


In [149]:
exp_time=5.0 # seconds

group_id=astropytime.Time.now().tai.isot

image_1 = await latiss.take_image(exptime=exp_time, shutter=True,image_type='OBJECT',
                                  group_id=group_id, filter='KPNO_406_828nm', grating='empty_1')# 'blank_bk7_wg05' KPNO_406_828nm'  'KPNO_1111_436nm' 'KPNO_373A_677nm' 'ronchi170lpmm'

print(image_1.imageName)

Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages
AT_O_20200128_000361


In [141]:
c.evt_logMessage.set_put(message="1 - Take reference image with no offset. [END]")

True

## 2 - Apply hexapod +0.5mm offset in the x-direction and take an exposure

In [134]:
c.evt_logMessage.set_put(message="2 - Apply hexapod +0.5mm offset in the x-direction and take an exposure [START]")

True

In [135]:
attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_offset.set_start(x=+0.5, y=0., z=0)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

In [136]:
group_id=astropytime.Time.now().tai.isot

image_2 = await latiss.take_image(exptime=exp_time, shutter=True,image_type='OBJECT',
                                  group_id=group_id, filter='KPNO_406_828nm', grating='empty_1')# 'blank_bk7_wg05' KPNO_406_828nm'  'KPNO_1111_436nm' 'KPNO_373A_677nm' 'ronchi170lpmm'

print(image_2.imageName)

Waiting for header service LFO before continuing
AT_O_20200128_000358


In [137]:
c.evt_logMessage.set_put(message="2 - Apply hexapod +0.5mm offset in the x-direction and take an exposure [END]")

True

## 3 - Zero offset in hexapod, do 30 arcsec offset in elevation and take an exposure

In [142]:
c.evt_logMessage.set_put(message="3 - Zero offset in hexapod, do 30 arcsec offset in elevation and take an exposure [START]")

True

In [143]:
attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_offset.set_start(x=-0.5, y=0., z=0)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)
attcs.atmcs.evt_allAxesInPosition.flush()
await attcs.atptg.cmd_offsetAzEl.set_start(az=60, el=20+30, num=0) #refence offset
try:
    await attcs.atmcs.evt_allAxesInPosition.next(flush=False, timeout=30.)
except asyncio.TimeoutError:
    print("error getting allAxesInPosition event.")

error getting allAxesInPosition event.


In [144]:
group_id=astropytime.Time.now().tai.isot

image_3 = await latiss.take_image(exptime=exp_time, shutter=True,image_type='OBJECT',
                                  group_id=group_id, filter='KPNO_406_828nm', grating='empty_1')# 'blank_bk7_wg05' KPNO_406_828nm'  'KPNO_1111_436nm' 'KPNO_373A_677nm' 'ronchi170lpmm'

print(image_3.imageName)

Waiting for header service LFO before continuing
AT_O_20200128_000360


In [145]:
c.evt_logMessage.set_put(message="3 - Zero offset in hexapod, do 30 arcsec offset in elevation and take an exposure [END]")

True

## 4 - apply hexapod +0.5mm offset in the y-direction and take an exposure

In [118]:
c.evt_logMessage.set_put(message="4 - apply hexapod +0.5mm offset in the y-direction and take an exposure [START]")

True

In [119]:
attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_offset.set_start(y=+0.5, x=0., z=0)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)
attcs.atmcs.evt_allAxesInPosition.flush()
await attcs.atptg.cmd_offsetAzEl.set_start(az=60, el=20, num=0) #refence offset
try:
    await attcs.atmcs.evt_allAxesInPosition.next(flush=False, timeout=30.)
except asyncio.TimeoutError:
    print("error getting allAxesInPosition event.")

In [120]:
group_id=astropytime.Time.now().tai.isot

image_4 = await latiss.take_image(exptime=exp_time, shutter=True,image_type='OBJECT',
                                  group_id=group_id, filter='KPNO_406_828nm', grating='empty_1')# 'blank_bk7_wg05' KPNO_406_828nm'  'KPNO_1111_436nm' 'KPNO_373A_677nm' 'ronchi170lpmm'

print(image_4.imageName)

Waiting for header service LFO before continuing
AT_O_20200128_000355


In [121]:
c.evt_logMessage.set_put(message="4 - apply hexapod +0.5mm offset in the y-direction and take an exposure [END]")

True

## 5 - Zero offset in hexapod, do 30 arcsec offset in azimuth and take an exposure

In [122]:
c.evt_logMessage.set_put(message="5 - Zero offset in hexapod, do 30 arcsec offset in azimuth and take an exposure [START]")

True

In [123]:
attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_offset.set_start(y=-0.5, x=0., z=0)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)
attcs.atmcs.evt_allAxesInPosition.flush()
await attcs.atptg.cmd_offsetAzEl.set_start(az=60+30, el=20, num=0) #refence offset
try:
    await attcs.atmcs.evt_allAxesInPosition.next(flush=False, timeout=30.)
except asyncio.TimeoutError:
    print("error getting allAxesInPosition event.")

In [124]:
group_id=astropytime.Time.now().tai.isot

image_5 = await latiss.take_image(exptime=exp_time, shutter=True,image_type='OBJECT',
                                  group_id=group_id, filter='KPNO_406_828nm', grating='empty_1')# 'blank_bk7_wg05' KPNO_406_828nm'  'KPNO_1111_436nm' 'KPNO_373A_677nm' 'ronchi170lpmm'

print(image_5.imageName)

Waiting for header service LFO before continuing
AT_O_20200128_000356


In [125]:
c.evt_logMessage.set_put(message="5 - Zero offset in hexapod, do 30 arcsec offset in azimuth and take an exposure [END]")

True

In [126]:
c.evt_logMessage.set_put(message="Test of axis orientation on the camera. [END]")

True

In [ ]:
rotation_matrix = lambda angle: np.array([
            [np.cos(np.radians(angle)), -np.sin(np.radians(angle)), 0.],
            [np.sin(np.radians(angle)), np.cos(np.radians(angle)), 0.],
            [0., 0., 1.]])

In [212]:
await attcs.atptg.cmd_offsetAzEl.set_start(az=0, el=0, num=0) #refence offset

In [213]:
group_id=astropytime.Time.now().tai.isot

image_5 = await latiss.take_image(exptime=exp_time, shutter=True,image_type='OBJECT',
                                  group_id=group_id, filter='KPNO_406_828nm', grating='empty_1')# 'blank_bk7_wg05' KPNO_406_828nm'  'KPNO_1111_436nm' 'KPNO_373A_677nm' 'ronchi170lpmm'

print(image_5.imageName)

Waiting for header service LFO before continuing
AT_O_20200128_000378


59.37620041807779 -51.17594221309642
8.200258204981367


In [ ]:
await attcs.atmcs.evt_azimuthInPosition

In [216]:
azel = await attcs.atmcs.tel_mount_AzEl_Encoders.aget()
nasmyth = await attcs.atmcs.tel_mount_Nasmyth_Encoders.aget()

print(np.mean(azel.elevationCalculatedAngle),np.mean(nasmyth.nasmyth2CalculatedAngle))
print(np.mean(azel.elevationCalculatedAngle)+np.mean(nasmyth.nasmyth2CalculatedAngle))


x, y, _ = np.matmul([140., 60., 0.], 
                    rotation_matrix(np.mean(azel.elevationCalculatedAngle)+np.mean(nasmyth.nasmyth2CalculatedAngle)-23.12084866242472))
print(x,y)

await attcs.atptg.cmd_offsetAzEl.set_start(az=x, el=y, num=0) #refence offset

60.077908711038184 -43.87625082061306
16.201657890425125
131.75222572666266 76.42873161364467


-45.56482150633152 39.03648346221966


In [217]:
group_id=astropytime.Time.now().tai.isot

image_5 = await latiss.take_image(exptime=exp_time, shutter=True,image_type='OBJECT',
                                  group_id=group_id, filter='KPNO_406_828nm', grating='empty_1')# 'blank_bk7_wg05' KPNO_406_828nm'  'KPNO_1111_436nm' 'KPNO_373A_677nm' 'ronchi170lpmm'

print(image_5.imageName)

Waiting for header service LFO before continuing
AT_O_20200128_000380
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 12 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 83 messages
atdometrajectory not in <State.ENABLED: 2>: <State.DISABLED: 1>
ataos not in <State.ENABLED: 2>: <State.DISABLED: 1>
atptg not in <State.ENABLED: 2>: <State.DISABLED: 1>
athexapod not in <State.ENABLED: 2>: <State.DISABLED: 1>
atdome not in <State.ENABLED: 2>: <State.DISABLED: 1>
atpneumatics not in <State.ENABLED: 2>: <State.DISABLED: 1>
